# Donut Fine-tuning

El presente notebook es el proceso de fine-tuning para [DoNut-base](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://huggingface.co/naver-clova-ix/donut-base-finetuned-cord-v2&ved=2ahUKEwjMh4O54vGNAxVsIrkGHQznKskQFnoECBcQAQ&usg=AOvVaw1uKtlO2jgCL6oC_haM4FIB).

In [1]:
pip install datasets transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
#from transformers import MobileViTForImageClassification, MobileViTImageProcessor
from transformers import ViTForImageClassification, ViTImageProcessor
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt
import numpy as np

2025-06-16 16:53:29.276969: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 16:53:29.286053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750103609.296095 1536958 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750103609.299149 1536958 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750103609.307939 1536958 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
device =  'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device) # Expected: ‘cuda’ if Linux else ‘mps’ if MacOS
device =  'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

NameError: name 'torch' is not defined

In [4]:
print(torch.__version__)

2.6.0+cu126


# Descargando modelos

In [2]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

img_dir_train = "../data/raw/subset/train2014/train2014"
img_dir_val = "../data/raw/subset/val2014"
img_dir_test = "../data/raw/subset/test2014"
ann_coco_text = "/home/juan/CEIA/CEIA-ViT/TrabajosPracticos/TP_Final/data/annotations/cocotext.v2.json"
# transform = transforms.Compose([
#     transforms.ToTensor()
# ])
# 
# coco_detection_train = CocoDetection(root=img_dir_train, annFile=ann_coco_text, transform=transform)
# coco_detection_val = CocoDetection(root=img_dir_val, annFile=ann_coco_text, transform=transform)

In [6]:
# from pycocotools.coco import COCO
# 
# # Load annotations
# coco_train = COCO(ann_file_train)
# coco_test = COCO(ann_file_test)
# coco_val = COCO(ann_file_val)

In [7]:
# # Get all image ids
# img_ids_train = coco_detection_train.getImgIds()
# img_data_train = coco_detection_train.loadImgs(img_ids_train[0])[0]
# 
# img_ids_val = coco_detection_val.getImgIds()
# img_data_val = coco_detection_val.loadImgs(img_ids_val[0])[0]


In [8]:
# # Pick the first image in COCO-Train that has scene text
# for img_id in coco_train.getImgIds():
#     if img_id in filtered_img_train:
#         selected_img_id = img_id
#         break


In [9]:
# import json
# 
# with open(ann_coco_text, 'r') as f:
#     coco_text_data = json.load(f)


In [10]:
# # Load COCO-Text annotations
# scene_text_ann_ids = coco_text_data["imgToAnns"].get(str(selected_img_id), [])
# scene_text_anns = [coco_text_data["anns"][str(ann_id)] for ann_id in scene_text_ann_ids]
# coco_info = coco_train.loadImgs(selected_img_id)[0]


In [4]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

# 1. First fix the model configuration
model.config.decoder_start_token_id = processor.tokenizer.pad_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
import json
import os
from PIL import Image
from torch.utils.data import Dataset
import torch

class DonutTextDatasetFromCocoTextV2Raw(Dataset):
    def __init__(self, image_dir, ann_file, processor, max_length=512):
        self.image_dir = image_dir
        self.processor = processor
        self.max_length = max_length

        with open(ann_file, "r") as f:
            coco_data = json.load(f)

        self.anns = coco_data["anns"]
        self.imgs = coco_data["imgs"]

        # Indexar anotaciones por imagen_id
        self.ann_by_image = {}
        for ann in self.anns.values():
            img_id = ann["image_id"]
            if img_id not in self.ann_by_image:
                self.ann_by_image[img_id] = []
            self.ann_by_image[img_id].append(ann)

        # Filtrar imágenes existentes en image_dir
        image_files = set(os.listdir(image_dir))
        self.valid_img_ids = [
            int(img_id)
            for img_id, img_data in self.imgs.items()
            if img_data["file_name"] in image_files
        ]

    def __len__(self):
        return len(self.valid_img_ids)

    def __getitem__(self, idx):
        img_id = self.valid_img_ids[idx]
        img_info = self.imgs[str(img_id)]
        img_path = os.path.join(self.image_dir, img_info["file_name"])

        image = Image.open(img_path).convert("RGB")

        # Obtener texto legible
        anns = self.ann_by_image.get(img_id, [])
        texts = [
            ann["utf8_string"] for ann in anns
            if ann.get("legibility", "") == "legible" and ann.get("utf8_string", "").strip() != ""
        ]
        text = " ".join(texts).strip()

        # 1. Imagen
        pixel_values = self.processor(image, return_tensors="pt")["pixel_values"].squeeze()
        
        # 2. Texto (tokenizado manualmente)
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        ).input_ids.squeeze()

        # 3. Reemplazo de tokens pad
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        return {
            "pixel_values": pixel_values,
            "labels": labels
        }

In [10]:


train_dataset = DonutTextDatasetFromCocoTextV2Raw(img_dir_train, ann_coco_text, processor=processor)
val_dataset = DonutTextDatasetFromCocoTextV2Raw(img_dir_val, ann_coco_text, processor=processor)
test_dataset = DonutTextDatasetFromCocoTextV2Raw(img_dir_test, ann_coco_text, processor=processor)


In [9]:
val_dataset.__len__()


7129

In [10]:
train_dataset.__len__()

15656

In [11]:
test_dataset.__len__()

700

In [27]:
sample = train_dataset[0]
first_image = sample["pixel_values"]  # This is the processed image tensor

In [ ]:
first_text = processor.tokenizer.decode(sample["labels"][train_dataset[0]["labels"] != -100])  # Decode the text labels

In [ ]:
from PIL import Image

# If you want to see the actual PIL image before processing:
first_image_path = os.path.join(img_dir_train, train_dataset.imgs[str(train_dataset.valid_img_ids[0])]["file_name"])
original_image = Image.open(first_image_path).convert("RGB")

# Display results
print("Detected Text:", first_text)
original_image.show()  # This will display the original image

### Inferencia con modelo preentrenado (sin finetuning)

In [ ]:
# prepare decoder inputs

task_prompt = "<s_cord-v2>"

decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(original_image, return_tensors="pt").pixel_values


In [ ]:
# Ensure model is on GPU
model = model.to('cuda')

# Move inputs to GPU
pixel_values = pixel_values.to('cuda')
decoder_input_ids = decoder_input_ids.to('cuda')
outputs = model.generate(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids)

result = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(result)

# Fine-tuning

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true" # no utilizamos weights and biases

In [6]:
training_args = TrainingArguments(
    output_dir="./donut-finetuned-coco2014",
    per_device_train_batch_size=1,  # Minimum possible
    gradient_accumulation_steps=1,  # No accumulation
    fp16=True,                      # Mixed precision
    gradient_checkpointing=True,    # Memory optimization
    optim="adamw_8bit",             # 8-bit optimizer
    eval_strategy="no",
    per_device_eval_batch_size=1,
    save_strategy="epoch",             # Disable checkpoints
    logging_steps=50,
    learning_rate=1e-5,
    num_train_epochs=1              # Start with 1 epoch
)

from jiwer import wer, cer

def compute_metrics(pred,donut_processor):
    # Extraer logits (shape: [batch, seq_len, vocab_size])
    pred_ids = pred.predictions.argmax(-1)

    # Decodificar predicciones y etiquetas
    pred_str = donut_processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = donut_processor.tokenizer.batch_decode(pred.label_ids, skip_special_tokens=True)

    # Limpiar cadenas (opcional, según formato de texto)
    pred_str = [s.strip() for s in pred_str]
    label_str = [s.strip() for s in label_str]

    return {
        "cer": cer(label_str, pred_str),
        "wer": wer(label_str, pred_str)
    }


In [9]:
from functools import partial

compute_metrics_bound = partial(compute_metrics, donut_processor=processor)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_bound,
)


NameError: name 'train_dataset' is not defined

### Probamos evaluar el modelo tal cual viene

In [12]:
import torch
from tqdm import tqdm

def manual_evaluate(model, dataset, processor, max_samples=None):
    model.eval()
    predictions, references = [], []
    max_samples = max_samples or len(dataset)
    empty_reference_count = 0
    correct_empty_predictions = 0
    
    for i in tqdm(range(max_samples)):
        try:
            # 1. Load sample
            sample = dataset[i]
            pixel_values = sample["pixel_values"].unsqueeze(0).to(model.device)
            
            # Ensure type matches model precision
            if model.dtype == torch.float16:
                pixel_values = pixel_values.half()
            # 2. Generate prediction
            with torch.no_grad():
                outputs = model.generate(
                    pixel_values,
                    max_length=512,
                    pad_token_id=processor.tokenizer.pad_token_id,
                    num_beams=1,
                )
            
            # 3. Decode prediction
            pred_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
            
            # 4. Safely decode labels
            valid_label_ids = sample["labels"][sample["labels"] != -100]  # Remove padding
            valid_label_ids = valid_label_ids[valid_label_ids < processor.tokenizer.vocab_size]  # Filter invalid IDs
            true_text = processor.decode(valid_label_ids, skip_special_tokens=True)
            # 4. Handle empty references separately
            if not true_text:
                empty_reference_count += 1
                if not pred_text:
                    correct_empty_predictions += 1
                continue  # Skip jiwer calculation for empty references


            predictions.append(pred_text.strip())
            references.append(true_text.strip())
            
        except Exception as e:
            print(f"Error processing sample {i}: {str(e)}")
            continue
            
        # Periodic cleanup
        if i % 10 == 0:
            torch.cuda.empty_cache()
    
    # Calculate standard metrics (only non-empty references)
    metrics = {
        "cer": cer(references, predictions) if references else float('nan'),
        "wer": wer(references, predictions) if references else float('nan'),
        "samples_evaluated": len(references) + empty_reference_count,
        "empty_reference_accuracy": correct_empty_predictions / empty_reference_count if empty_reference_count > 0 else float('nan'),
        "empty_references": empty_reference_count,
        "correct_empty_predictions": correct_empty_predictions
    }
    
    return metrics
# Usage

In [11]:
model.eval()  # Set to evaluation mode
torch.cuda.empty_cache()

metrics = manual_evaluate(
    model=model,
    dataset=test_dataset,
    processor=processor,
    max_samples=None  # Start small, then increase if successful
)
print(metrics)

100%|██████████| 700/700 [02:34<00:00,  4.54it/s]


NameError: name 'cer' is not defined

In [20]:
torch.cuda.empty_cache() # limpiamos cache

# Entrenamiento

In [10]:
trainer.train()
# trainer.train(resume_from_checkpoint=True)  # Automatically finds latest checkpoint

`use_cache=True` is incompatible with gradient checkpointing`. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,4.907700
100,4.609500
150,3.504500
200,3.620100
250,3.930400
300,3.548000
350,3.050400
400,3.834800
450,2.674900
500,3.629700


TrainOutput(global_step=15656, training_loss=2.9239136513606225, metrics={'train_runtime': 7675.8143, 'train_samples_per_second': 2.04, 'train_steps_per_second': 2.04, 'total_flos': 4.8954928808381645e+19, 'train_loss': 2.9239136513606225, 'epoch': 1.0})

In [11]:
trainer.save_model("./donut-finetuned-coco2014/final-model")
processor.save_pretrained("./donut-finetuned-coco2014/final-model")


[]

# Evaluamos modelo finetuneado para ver si mejoro alguna metrica

In [8]:
import torch
import glob
from transformers import VisionEncoderDecoderModel, DonutProcessor

torch.cuda.empty_cache()

checkpoints = glob.glob("./donut-finetuned-coco2014/final-model/")
# assert checkpoints, "No checkpoints found!"
# latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[-1]))

processor = DonutProcessor.from_pretrained("./donut-finetuned-coco2014/final-model/")

model = VisionEncoderDecoderModel.from_pretrained(
    "./donut-finetuned-coco2014/final-model/",
    device_map="auto",
    torch_dtype=torch.float16  
)


test_dataset = DonutTextDatasetFromCocoTextV2Raw(
    img_dir_test,  
    ann_coco_text,
    processor=processor,
    max_length=512
)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [17]:
print(f"Model device: {model.device}")
print(f"Vocab size: {model.config}")

Model device: cuda:0
Vocab size: VisionEncoderDecoderConfig {
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "decoder": {
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "attention_dropout": 0.0,
    "classifier_dropout": 0.0,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "dropout": 0.1,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "encoder_layerdrop": 0.0,
    "encoder_layers": 12,
    "init_std": 0.02,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "max_position_embeddings": 768,
    "model_type": "mbart",
    "num_hidden_layers": 12,
    "scale_embedding": true,
    "torch_dtype": "float16",
    "use_cache": true,
    "vocab_size": 57580
  },
  "decoder_start_token_id": 1,
  "encoder": {
    "attention_probs_dropout_prob": 0.0,
    "depths": [
      

In [13]:
metrics = manual_evaluate(
    model=model,
    dataset=test_dataset,
    processor=processor,
    max_samples=None  
)
print(metrics)

100%|██████████| 700/700 [02:30<00:00,  4.65it/s]

{'cer': 6.245715350223547, 'wer': 0.9957122439256789, 'samples_evaluated': 700, 'empty_reference_accuracy': 0.9638009049773756, 'empty_references': 221, 'correct_empty_predictions': 213}


In [ ]:
model

# Resultados